In [45]:
import pandas as pd
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, accuracy_score,classification_report

In [36]:
df = pd.read_csv('train.csv')
df.dropna()
df.reset_index()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...,...
20795,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [37]:
# ### Replace Punctuations
# X.replace("[^a-zA-Z]", ' ', regex=True,inplace=True)
# ### Convert to lowercase
# for index in X.columns:
#     X[index] = X[index].str.lower()
# Convert to a paragraph
# headlines = []
# for row in range(0,len(X.index)):
#     headlines.append(' '.join(str(x) for x in X.iloc[row,0:25]))

ps = PorterStemmer()
corpus = []
for i in range(0,len(X)):
    review = re.sub('[^a-z-A-Z]', ' ', str(df['title'][i]))
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

### Bag Of Words Model

In [41]:
cv = CountVectorizer(max_features=5000 , ngram_range=(1,3))
X = cv.fit_transform(corpus).toarray()

In [42]:
y = df['label']

In [43]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

### Naive Bayes

In [46]:
classifier = MultinomialNB()
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)

In [47]:
cm = confusion_matrix(y_test,y_pred)
acc = accuracy_score(y_test,y_pred)
print(cm,acc)

[[1831  215]
 [ 197 1917]] 0.9009615384615385


### Random Forest

In [48]:
classifier = RandomForestClassifier(n_estimators=200,criterion='entropy')
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)

In [49]:
cm = confusion_matrix(y_test,y_pred)
acc = accuracy_score(y_test,y_pred)
print(cm,acc)

[[1846  200]
 [  54 2060]] 0.9389423076923077


### TFID Vectorizer

In [54]:
cv = TfidfVectorizer()
X = cv.fit_transform(corpus).toarray()

In [55]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [56]:
classifier = MultinomialNB()
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test,y_pred)
acc = accuracy_score(y_test,y_pred)
print(cm,acc)

[[1957   89]
 [ 433 1681]] 0.8745192307692308


In [57]:
classifier = RandomForestClassifier(n_estimators=200,criterion='entropy')
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test,y_pred)
acc = accuracy_score(y_test,y_pred)
print(cm,acc)

[[1845  201]
 [  38 2076]] 0.942548076923077
